**Feature Selection**: RF and Chi Squared Revisited
------------------

In [1]:
# https://infosecjupyterthon.com/notebooks/2_Jupyterthon_Cyb3rPandaH_2020.html
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import length, concat ,concat_ws

import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
import requests
from openhunt import ossem, descriptiveStatistics as ds, visualizations as vis
import warnings
warnings.filterwarnings('ignore')

# Import packages
import os
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.sparkContext.setCheckpointDir("./tmp/checkpoints")
spark.sparkContext.version
spark.conf.set("spark.driver.memory", "3g")
spark.conf.set("spark.executor.memory", "3g")

In [3]:
df_index = spark.read.json('df_labelled_coalesce.json')

In [4]:
df_index = df_index.select("*").withColumn("id", func.monotonically_increasing_id()).drop("tags")

In [5]:
df_index = df_index.withColumn("TP_Binary", func.when((df_index["TP_Label"]=='0'), 0).otherwise(1))

In [6]:
df_index = df_index.checkpoint()

In [7]:
df_index = df_index.cache()

In [8]:
df_index.createTempView('apt29Host')

In [9]:
target_host_day1 = spark.sql(
'''
SELECT *
FROM apt29Host
WHERE NOT TP_Label = '0'
''')

In [10]:
pd_target_distict_count = target_host_day1.agg(*[
    func.countDistinct(c).alias(c)    # vertical (column-wise) operations in SQL ignore NULLs
    for c in target_host_day1.columns
]).toPandas().transpose()

In [11]:
pd.options.display.max_rows = 100
pd_target_distict_count.sort_values(by = 0, ascending = False).head(80)

,0
id,3438
RecordNumber,759
Message,756
ActivityID,638
ContextInfo,623
@timestamp,475
Payload,416
EventReceivedTime,87
EventTime,78
UtcTime,59


In [14]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [15]:
def ExtractFeatureSelection(featuresSelected, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
      list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(featuresSelected).rename(columns={0:"idx"})
    pd_list_extract = pd.DataFrame(list_extract)
    varlist['name'] = varlist['idx'].apply(lambda x: pd_list_extract.iloc[x, 1])
    return(varlist)

In [16]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import ChiSqSelector

In [17]:
# RF Feature Importance
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["EventID", "SourceName"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8888,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(20)

,idx,name,score
0,0,OHE_EventID_12,0.0
124,124,OHE_EventID_6005,0.0
126,126,OHE_EventID_306,0.0
127,127,OHE_EventID_5033,0.0
128,128,OHE_EventID_1074,0.0
129,129,OHE_EventID_1136,0.0
130,130,OHE_EventID_7002,0.0
131,131,OHE_EventID_51047,0.0
132,132,OHE_EventID_21,0.0
133,133,OHE_EventID_4776,0.0


In [18]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["EventID", "SourceName"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
selector = ChiSqSelector(numTopFeatures=50, featuresCol='features', outputCol="selectedFeatures", labelCol='label', selectorType='numTopFeatures', percentile=0.1, fpr=0.05, fdr=0.05, fwe=0.05)

In [19]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, selector])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)

In [20]:
ExtractFeatureSelection(mod.stages[-1].selectedFeatures, df2, "features")

,idx,name
0,0,OHE_EventID_12
1,1,OHE_EventID_10
2,2,OHE_EventID_7
3,3,OHE_EventID_13
4,4,OHE_EventID_4658
5,5,OHE_EventID_4103
6,6,OHE_EventID_4656
7,7,OHE_EventID_4690
8,8,OHE_EventID_4663
9,9,OHE_EventID_800


In [22]:
# RF Feature Importance
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["Hashes", "Image", "ProcessGuid", "ParentProcessGuid", "ProviderGuid", "ThreadID", "ActivityID", "ExecutionProcessID", "LogonGuid", "SubjectUserSid", "TargetUserSid", "TargetSid", "UserSid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8888,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(20)

,idx,name,score
8228,8228,OHE_ExecutionProcessID_3876,0.098002
3039,3039,OHE_ActivityID_{BCCD0404-4290-0000-613D-11D71F...,0.083050
3033,3033,OHE_ActivityID_{BCCD0404-4290-0001-7880-12D71F...,0.065133
3032,3032,OHE_ActivityID_{BCCD0404-4290-0001-74B4-12D71F...,0.062878
3012,3012,OHE_ActivityID_{BCCD0404-4290-0000-8470-11D71F...,0.051422
3030,3030,OHE_ActivityID_{BCCD0404-4290-0001-DEA9-12D71F...,0.041828
3147,3147,OHE_ActivityID_{BCCD0404-4290-0000-B87A-11D71F...,0.037540
3273,3273,OHE_ActivityID_{BCCD0404-4290-0001-58A0-12D71F...,0.029273
3188,3188,OHE_ActivityID_{BCCD0404-4290-0000-3243-11D71F...,0.027911
3093,3093,OHE_ActivityID_{BCCD0404-4290-0000-5945-11D71F...,0.025714


In [24]:
pd.options.display.max_colwidth = 100
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(100)

,idx,name,score
8228,8228,OHE_ExecutionProcessID_3876,0.098002
3039,3039,OHE_ActivityID_{BCCD0404-4290-0000-613D-11D71F20D601},0.083050
3033,3033,OHE_ActivityID_{BCCD0404-4290-0001-7880-12D71F20D601},0.065133
3032,3032,OHE_ActivityID_{BCCD0404-4290-0001-74B4-12D71F20D601},0.062878
3012,3012,OHE_ActivityID_{BCCD0404-4290-0000-8470-11D71F20D601},0.051422
3030,3030,OHE_ActivityID_{BCCD0404-4290-0001-DEA9-12D71F20D601},0.041828
3147,3147,OHE_ActivityID_{BCCD0404-4290-0000-B87A-11D71F20D601},0.037540
3273,3273,OHE_ActivityID_{BCCD0404-4290-0001-58A0-12D71F20D601},0.029273
3188,3188,OHE_ActivityID_{BCCD0404-4290-0000-3243-11D71F20D601},0.027911
3093,3093,OHE_ActivityID_{BCCD0404-4290-0000-5945-11D71F20D601},0.025714


In [25]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["Hashes", "Image", "ProcessGuid", "ParentProcessGuid", "ProviderGuid", "ThreadID", "ActivityID", "ExecutionProcessID", "LogonGuid", "SubjectUserSid", "TargetUserSid", "TargetSid", "UserSid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
selector = ChiSqSelector(numTopFeatures=50, featuresCol='features', outputCol="selectedFeatures", labelCol='label', selectorType='numTopFeatures', percentile=0.1, fpr=0.05, fdr=0.05, fwe=0.05)

In [26]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, selector])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)

In [27]:
ExtractFeatureSelection(mod.stages[-1].selectedFeatures, df2, "features")

,idx,name
0,123,"OHE_Hashes_SHA1=36C5D12033B2EAF251BAE61C00690FFB17FDDC87,MD5=CDA48FC75952AD12D99E526D0B6BF70A,SH..."
1,237,"OHE_Hashes_SHA1=7662A8D2F23C3474DEC6EF8E2B0365B0B86714EE,MD5=F68AF942FD7CCC0E7BAB1A2335D2AD26,SH..."
2,276,"OHE_Hashes_SHA1=585EB59D12A111E9291518C5CF5D3FD296C2B581,MD5=57292CE8714E2D221D9D97C9D061D332,SH..."
3,1741,OHE_Image_C:\windows\system32\svchost.exe
4,1742,OHE_Image_C:\windows\system32\WindowsPowerShell\v1.0\powershell.exe
5,1743,OHE_Image_C:\windows\system32\wbem\wmiprvse.exe
6,1744,OHE_Image_C:\Windows\System32\svchost.exe
7,1745,OHE_Image_C:\windows\system32\backgroundTaskHost.exe
8,1746,OHE_Image_C:\windows\System32\svchost.exe
9,1747,OHE_Image_C:\windows\system32\sihost.exe


In [28]:
# RF Feature Importance
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["Hashes", "Image", "ProcessGuid", "ParentProcessGuid", "ProviderGuid", "ThreadID", "ActivityID", "ExecutionProcessID", "LogonGuid", "SubjectUserSid", "TargetUserSid", "TargetSid", "UserSid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8888,
                            numTrees=30, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)

In [29]:
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(100)

,idx,name,score
2748,2748,OHE_ThreadID_5584,0.097957
8228,8228,OHE_ExecutionProcessID_3876,0.033183
2714,2714,OHE_ProviderGuid_{A0C1853B-5C40-4B15-8766-3CF1C58F985A},0.032312
3020,3020,OHE_ActivityID_{BCCD0404-4290-0001-A27F-12D71F20D601},0.030968
3017,3017,OHE_ActivityID_{BCCD0404-4290-0000-2C54-11D71F20D601},0.028199
3021,3021,OHE_ActivityID_{BCCD0404-4290-0001-FFC8-12D71F20D601},0.024643
3012,3012,OHE_ActivityID_{BCCD0404-4290-0000-8470-11D71F20D601},0.024464
8221,8221,OHE_ExecutionProcessID_3368,0.021720
3260,3260,OHE_ActivityID_{BCCD0404-4290-0000-1072-11D71F20D601},0.017677
3216,3216,OHE_ActivityID_{BCCD0404-4290-0001-59C9-12D71F20D601},0.017593


In [30]:
# RF Feature Importance
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["Hashes", "Image", "ProcessGuid", "ParentProcessGuid", "ProviderGuid", "ThreadID", "ActivityID", "ExecutionProcessID", "LogonGuid", "SubjectUserSid", "TargetUserSid", "TargetSid", "UserSid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8888,
                            numTrees=60, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)

In [31]:
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(100)

,idx,name,score
8228,8228,OHE_ExecutionProcessID_3876,0.033049
2748,2748,OHE_ThreadID_5584,0.032959
2714,2714,OHE_ProviderGuid_{A0C1853B-5C40-4B15-8766-3CF1C58F985A},0.032346
3027,3027,OHE_ActivityID_{BCCD0404-4290-0001-03BF-12D71F20D601},0.023215
3030,3030,OHE_ActivityID_{BCCD0404-4290-0001-DEA9-12D71F20D601},0.022925
3023,3023,OHE_ActivityID_{BCCD0404-4290-0001-73D3-12D71F20D601},0.020730
3018,3018,OHE_ActivityID_{BCCD0404-4290-0000-D47C-11D71F20D601},0.018039
3026,3026,OHE_ActivityID_{BCCD0404-4290-0000-ED54-11D71F20D601},0.017358
3020,3020,OHE_ActivityID_{BCCD0404-4290-0001-A27F-12D71F20D601},0.015324
3037,3037,OHE_ActivityID_{BCCD0404-4290-0001-EEBE-12D71F20D601},0.013491


In [32]:
columnList = ["Hashes", "Image", "ProcessGuid", "ParentProcessGuid", "ProviderGuid", "ThreadID", "ActivityID", "ExecutionProcessID", "LogonGuid", "SubjectUserSid", "TargetUserSid", "TargetSid", "UserSid"]
# Categorical Freq Count by Columns with more than one distinct value in the target subset
for i in columnList:
    display(target_host_day1.groupBy(target_host_day1[i]).count().orderBy('count', ascending=False).toPandas())

,Hashes,count
0,None,3405
1,"SHA1=36C5D12033B2EAF251BAE61C00690FFB17FDDC87,MD5=CDA48FC75952AD12D99E526D0B6BF70A,SHA256=908B64...",7
2,"SHA1=7662A8D2F23C3474DEC6EF8E2B0365B0B86714EE,MD5=F68AF942FD7CCC0E7BAB1A2335D2AD26,SHA256=11064E...",5
3,"SHA1=585EB59D12A111E9291518C5CF5D3FD296C2B581,MD5=57292CE8714E2D221D9D97C9D061D332,SHA256=43782E...",4
4,"SHA1=7BCD946326B67F806B3DB4595EDE9FBDF29D0C36,MD5=2B5CB081721B8BA454713119BE062491,SHA256=FEEC14...",2
5,"SHA1=8DCA9749CD48D286950E7A9FA1088C937CBCCAD4,MD5=D7AB69FAD18D4A643D84A271DFC0DBDF,SHA256=FF79D3...",2
6,"SHA1=408238F3BEA1DF74E8B9B672E8F95C5BA2C5DBC0,MD5=FD021D31F1DFA5E00EFA035758023064,SHA256=8FD967...",1
7,"SHA1=691E81A8FA152F68FB8ACEFE8F59EA41DC995880,MD5=44F96457ADEB95AFD3F5457082D44538,SHA256=3247D2...",1
8,"SHA1=FF57080E5B19C3DCF62A00EA4037330F78B228CB,MD5=1B94BED747B7539280C9762C9C1B27EB,SHA256=8BE078...",1
9,"SHA1=4B7FA56A4E85F88B98D11A6E018698AE3FBA5E62,MD5=9D1C5EF38E6073661C74660B3A71A76E,SHA256=0DF38A...",1


,Image,count
0,None,3362
1,C:\Program Files\SysinternalsSuite\sdelete64.exe,18
2,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,17
3,C:\windows\system32\services.exe,7
4,C:\Windows\Temp\python.exe,6
5,C:\Windows\System32\rundll32.exe,5
6,C:\ProgramData\victim\â€®cod.3aka3.scr,4
7,C:\windows\System32\WindowsPowerShell\v1.0\powershell.exe,4
8,C:\windows\system32\WindowsPowerShell\v1.0\powershell.exe,3
9,C:\Windows\Temp\sdelete64.exe,3


,ProcessGuid,count
0,None,3362
1,{47ab858c-e374-5eac-d803-000000000400},9
2,{47ab858c-cad9-5eac-0b00-000000000400},7
3,{47ab858c-e305-5eac-d303-000000000400},6
4,{47ab858c-e2ac-5eac-cb03-000000000400},6
5,{47ab858c-e2f2-5eac-d203-000000000400},6
6,{47ab858c-e23d-5eac-c603-000000000400},4
7,{47ab858c-e13c-5eac-a903-000000000400},4
8,{47ab858c-e1e4-5eac-b803-000000000400},3
9,{47ab858c-e14e-5eac-ac03-000000000400},2


,ParentProcessGuid,count
0,None,3415
1,{47ab858c-e43f-5eac-eb03-000000000400},5
2,{47ab858c-e13c-5eac-a903-000000000400},2
3,{47ab858c-e1e4-5eac-b803-000000000400},2
4,{47ab858c-e188-5eac-b003-000000000400},2
5,{47ab858c-dac4-5eac-f202-000000000400},1
6,{5aa8ec29-e56d-5eac-6d03-000000000400},1
7,{5aa8ec29-e5e4-5eac-7a03-000000000400},1
8,{47ab858c-e23d-5eac-c603-000000000400},1
9,{5aa8ec29-e5b8-5eac-7903-000000000400},1


,ProviderGuid,count
0,{A0C1853B-5C40-4B15-8766-3CF1C58F985A},3307
1,{5770385F-C22A-43E0-BF4C-06F5698FFBD9},78
2,{54849625-5478-4994-A5BA-3E3B0328C30D},53


...

,LogonGuid,count
0,None,3412
1,{47ab858c-dabe-5eac-812e-370000000000},10
2,{47ab858c-dabe-5eac-f331-370000000000},6
3,{5aa8ec29-e5b7-5eac-7d17-890000000000},3
4,{d2e3bf90-d0c7-9b80-942b-c7b9cbec384a},3
5,{5aa8ec29-e4ec-5eac-2578-860000000000},1
6,{5aa8ec29-e52e-5eac-9ab5-860000000000},1
7,{5aa8ec29-e56d-5eac-53e3-860000000000},1
8,{47ab858c-e6ad-5eac-e703-000000000000},1


,SubjectUserSid,count
0,None,3394
1,S-1-5-21-1830255721-3727074217-2423397540-1107,35
2,S-1-5-18,6
3,S-1-0-0,3


,TargetUserSid,count
0,None,3414
1,S-1-0-0,16
2,S-1-5-21-1830255721-3727074217-2423397540-1107,8


,TargetSid,count
0,None,3438


,UserSid,count
0,None,3438


In [45]:
columnList = ["Protocol", "IpPort", "IpAddress", "SourcePort", "DestPort", "DestinationPort", "DestAddress", "DestinationIp"]
# Categorical Freq Count by Columns with more than one distinct value in the target subset
for i in columnList:
    display(target_host_day1.where(target_host_day1[i].isNotNull()).groupBy(['EventID', i]).count().orderBy('count', ascending=False).toPandas())

,EventID,Protocol,count
0,5156,6,9
1,3,tcp,9


,EventID,IpPort,count
0,5145,59987,4
1,5145,59997,4
2,5145,60018,4
3,5145,59976,4
4,5145,59967,3
5,4624,59967,3


,EventID,IpAddress,count
0,5145,10.0.1.4,19
1,4624,10.0.1.4,3


,EventID,SourcePort,count
0,3,59955,1
1,5156,59835,1
2,5156,59955,1
3,3,59961,1
4,5156,59846,1
5,5156,59960,1
6,3,59846,1
7,5156,59961,1
8,5156,59959,1
9,3,59960,1


,EventID,DestPort,count
0,5156,5985,4
1,5156,389,2
2,5156,443,2
3,5156,1234,1


,EventID,DestinationPort,count
0,3,5985,4
1,3,389,2
2,3,443,2
3,3,1234,1


,EventID,DestAddress,count
0,5156,10.0.1.6,4
1,5156,192.168.0.5,3
2,5156,10.0.0.4,2


,EventID,DestinationIp,count
0,3,10.0.1.6,4
1,3,192.168.0.5,3
2,3,10.0.0.4,2


In [40]:
target_host_day1.where(target_host_day1['DestAddress'].isNotNull()).groupBy(['EventID', 'DestAddress', 'DestPort']).count().orderBy('count', ascending=False).toPandas()

,EventID,DestAddress,DestPort,count
0,5156,10.0.1.6,5985,4
1,5156,192.168.0.5,443,2
2,5156,10.0.0.4,389,2
3,5156,192.168.0.5,1234,1


In [39]:
target_host_day1.where(target_host_day1['DestinationIp'].isNotNull()).groupBy(['EventID', 'DestinationIp', 'DestinationPort']).count().orderBy('count', ascending=False).toPandas()

,EventID,DestinationIp,DestinationPort,count
0,3,10.0.1.6,5985,4
1,3,192.168.0.5,443,2
2,3,10.0.0.4,389,2
3,3,192.168.0.5,1234,1


In [46]:
target_host_day1.where(target_host_day1['IpAddress'].isNotNull()).groupBy(['EventID', 'IpAddress', 'IpPort']).count().orderBy('count', ascending=False).toPandas()

,EventID,IpAddress,IpPort,count
0,5145,10.0.1.4,60018,4
1,5145,10.0.1.4,59997,4
2,5145,10.0.1.4,59987,4
3,5145,10.0.1.4,59976,4
4,4624,10.0.1.4,59967,3
5,5145,10.0.1.4,59967,3
